# Ads Recommendations System- Project


In [15]:
import pandas as pd

# Understanding the Problem Statement

In [31]:
user_data = pd.read_csv(r"C:\Users\beni computer\Desktop\Geeks_lab\week 5\user_data.csv")
user_data.head()

,event_time,user_id,event,channel,user_lat,user_long,origin,ad_id,images_count,ad_impressions,ad_views,ad_messages
0,2017-05-19 10:47:22,541,view,ios,-34.600439,-58.514031,browse_search,2426321,6.0,0.0,4.0,0.0
1,2017-06-10 10:18:12,501,view,android,-34.818047,-58.356583,browse_search,2746814,5.0,152.0,85.0,1.0
2,2017-06-11 00:44:57,559,view,android,-34.688454,-58.333435,browse,1123948,4.0,899.0,22.0,1.0
3,2017-05-22 02:36:06,637,first_message,android,-34.548401,-58.485168,search,2273498,NaN,0.0,21.0,2.0
4,2017-06-08 15:45:35,509,view,android,-34.582088,-58.405041,home,2722493,3.0,1529.0,132.0,3.0


In [17]:
user_messages = pd.read_csv(r"C:\Users\beni computer\Desktop\Geeks_lab\week 5\user_messages.csv")
user_messages.head()

,user_id,category_id,ads
0,1,859,[1806476]
1,3,800,"[2131700, 2734107, 2877209, 2877209]"
2,4,815,[2883211]
3,7,815,"[2429412, 2886810, 2886804]"
4,7,362,[2909301]


In [36]:
ads_data = pd.read_csv(r"C:\Users\beni computer\Desktop\Geeks_lab\week 5\ads_data.csv")
ads_data.head()

,ad_id,category_id,seller_id,creation_time,title,description,price,lat,long,source,enabled
0,1344,800,1157,2015-11-29 13:05:26,Blackberry curve,"Funcionando, es para personal el único problem...",300.0,NaN,NaN,android,0
1,3916,815,59,2015-12-16 09:09:25,Remera marca premium alemana,Casi nuevo y sin uso. Lo compre online y me qu...,150.0,NaN,NaN,android,1
2,11738,806,12298,2016-01-19 18:38:24,Juego de sillones estilo campo,Divino juego de sillones de diseño campo,1500.0,NaN,NaN,android,1
3,14939,815,1349,2016-01-30 23:47:30,Zapatillas,Zapatillas nro 39 dos usos,250.0,NaN,NaN,android,0
4,16820,815,16365,2016-02-05 10:22:29,Vestido de seda negro,Vestido importado de usa. Muy fino y elegante ...,500.0,NaN,NaN,android,0


# Recommending most popular elements

In [35]:
df = user_data[['ad_id','ad_messages']].groupby('ad_id', as_index=False).agg('count')
most_popular_ads = df.nlargest(10,['ad_messages'])['ad_id'].tolist()

In [20]:
sub1 = user_messages 
sub1['recommend'] = str(most_popular_ads)
sub1.head()

,user_id,category_id,ads,recommend
0,1,859,[1806476],"[2804693, 2794985, 2809690, 2814436, 26400, 28..."
1,3,800,"[2131700, 2734107, 2877209, 2877209]","[2804693, 2794985, 2809690, 2814436, 26400, 28..."
2,4,815,[2883211],"[2804693, 2794985, 2809690, 2814436, 26400, 28..."
3,7,815,"[2429412, 2886810, 2886804]","[2804693, 2794985, 2809690, 2814436, 26400, 28..."
4,7,362,[2909301],"[2804693, 2794985, 2809690, 2814436, 26400, 28..."


# Model Evaluation

In [37]:
def evaluate_accuracy(df):
    
    count = 0
    for index, row in df.iterrows():
        ads = row['ads'].replace('[','').replace(']','').split(',')
        recommended_ads = row['recommend'].replace('[','').replace(']','').split(',')
        
        for ad in ads:
            if (ad in recommended_ads):
                count = count + 1
                
    return count
                

In [38]:
def evaluate_precision(df):
    
    score = 0
    for index, row in df.iterrows():
        count = 0
        ads = row['ads'].replace('[','').replace(']','').split(',')
        recommended_ads = row['recommend'].replace('[','').replace(']','').split(',')
        
        for i in range(0, len(recommended_ads)):
            if (recommended_ads[i] in ads):
                count = count + 1
                score = score + count
        
        #print(ads, ":", recommended_ads,":" ,score)
        score = 1.0*score/len(ads)
        
                
    return score
                

In [39]:
print('accuracy of model 1 : ',evaluate_accuracy(sub1))
print('precision of model 1 : ',evaluate_precision(sub1))

accuracy of model 1 :  38
precision of model 1 :  8.356321959541373e-22


# Recommending category-wise most popular elements

In [41]:
df = user_data[['ad_id','ad_views']].groupby('ad_id', as_index=False).agg('sum')
most_popular_ads = df.nlargest(10,['ad_views'])['ad_id'].tolist()

In [42]:
df1 = user_data[['ad_id','ad_views']]
df2 = ads_data[['ad_id','category_id']]
merged_data = pd.merge(df1, df2, on='ad_id')

In [43]:
most_popular_ads_catwise = merged_data.groupby(['category_id','ad_id'], as_index=False).agg('count')

In [27]:
df = most_popular_ads_catwise
cats = df.category_id.unique()
cats

array([362, 800, 806, 811, 815, 853, 859, 881, 887, 888], dtype=int64)

In [28]:
cat_dict = {}
for cat in cats:
    df1 = df[df['category_id']==cat]
    cat_dict[cat] = df1.nlargest(10,['ad_views'])['ad_id'].tolist()
    
print(cat_dict)

{362: [2757104, 2836886, 2853312, 2560201, 2449079, 2806018, 2598159, 1704736, 2661821, 333573], 800: [2814436, 2806603, 2816630, 2816336, 2802819, 2031170, 2739908, 2649805, 2587910, 2737601], 806: [2804693, 2809690, 26400, 2825119, 2788868, 2764905, 2785533, 2516918, 2822691, 2336966], 811: [2626729, 2764838, 1928465, 2753896, 2597250, 2727592, 2839093, 2670023, 2746309, 2667189], 815: [2234932, 2434247, 2796911, 2812937, 2812849, 2719785, 2342155, 2816482, 2748424, 2434257], 853: [2853382, 2839832, 2082120, 2564683, 1190194, 2613818, 2632665, 2774269, 2832788, 2813016], 859: [2449447, 2567368, 2426549, 2274484, 1998464, 2740230, 2067241, 2778369, 1065275, 2842601], 881: [2794985, 2816864, 2817712, 2840833, 2727695, 2519508, 2662331, 2767340, 2846976, 2688815], 887: [2852082, 2677927, 2802433, 2634993, 2696625, 2714941, 2592955, 2810869, 1699375, 2031454], 888: [2254453, 2827386, 2740665, 2578937, 2826518, 2808111, 2826051, 2808047, 2808064, 2827584]}


In [29]:
def recommend(row):
    return str(cat_dict[row['category_id']])

sub2 = user_messages 
sub2['recommend'] = sub2.apply(recommend, axis = 1)
sub2.head()

,user_id,category_id,ads,recommend
0,1,859,[1806476],"[2449447, 2567368, 2426549, 2274484, 1998464, ..."
1,3,800,"[2131700, 2734107, 2877209, 2877209]","[2814436, 2806603, 2816630, 2816336, 2802819, ..."
2,4,815,[2883211],"[2234932, 2434247, 2796911, 2812937, 2812849, ..."
3,7,815,"[2429412, 2886810, 2886804]","[2234932, 2434247, 2796911, 2812937, 2812849, ..."
4,7,362,[2909301],"[2757104, 2836886, 2853312, 2560201, 2449079, ..."


In [45]:
print('accuracy of model 2 : ',evaluate_accuracy(sub2))
print('precision of model 2 : ',evaluate_precision(sub2))

accuracy of model 2 :  38
precision of model 2 :  8.356321959541373e-22
